In [34]:
# %load ../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import scale
from sklearn.preprocessing import PolynomialFeatures as pf
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from scipy.cluster import hierarchy
from sklearn.model_selection import cross_val_predict
import sklearn.model_selection as ms

import statsmodels.api as sm

import re

%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [7]:
bloombergNeg = pd.read_csv("data/labels/bloomberg_labels/bloomberg_neg_train.csv")
bloombergPos = pd.read_csv("data/labels/bloomberg_labels/bloomberg_pos_train.csv")
bloombergNeu = pd.read_csv("data/labels/bloomberg_labels/bloomberg_neu_train.csv")
bloombergTrain = pd.concat([bloombergNeg, bloombergPos, bloombergNeu], ignore_index=True)
bloombergTrain = bloombergTrain[['text', 's_label']]

In [9]:
sandersNeg = pd.read_csv("data/labels/sanders_labels/sanders_neg_train.csv")
sandersPos = pd.read_csv("data/labels/sanders_labels/sanders_pos_train.csv")
sandersNeu = pd.read_csv("data/labels/sanders_labels/sanders_neu_train.csv")
sandersTrain = pd.concat([sandersNeg, sandersPos, sandersNeu], ignore_index=True)
sandersTrain = sandersTrain[['text', 's_label']]

In [14]:
buttigegNeg = pd.read_csv("data/labels/buttigieg_labels/buttigieg_neg_train.csv")
buttigegPos = pd.read_csv("data/labels/buttigieg_labels/buttigieg_pos_train.csv")
buttigegNeu = pd.read_csv("data/labels/buttigieg_labels/buttigieg_neu_train.csv")
buttigiegTrain = pd.concat([buttigegNeg, buttigegPos, buttigegNeu], ignore_index=True)
buttigiegTrain = buttigieg[['text', 's_label']]

In [52]:
buttigiegFull = pd.read_csv("data/tweets_full/buttigieg_full.csv")
buttigiegFull = buttigiegFull[['text']]

In [53]:
buttigiegFull

,text
0,Pete Buttigieg is James Madison reincarnated
1,"More Democrats like this, please: ""A letter fr..."
2,#Bernie fan 16 yrs before it was cool. \nProgr...
3,@AbiWilks I like him but I'm pulling for longs...
4,I have such a crush on Pete Buttigieg.
...,...
6156,@DavidCornDC @ewarren Well she wasn't the smar...
6157,1. Elizabeth Warren\n2. Julian Castro\n3. Bern...
6158,Pete Buttigieg_Thank-You to Supporters https:/...
6159,Pete Buttigieg Birthday Party in the Cedar Rap...


In [19]:
def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df

In [23]:
bloombergTrain = clean_text(bloombergTrain, "text")
buttigiegTrain = clean_text(buttigiegTrain, "text")
sandersTrain = clean_text(sandersTrain, "text")

In [26]:
sandersTrain

,text,s_label
0,bernie sanders campaign accessed hillary clint...,-1
1,bernie sanders staffer playing dirty against h...,-1
2,bernie sanders will not get us free college im...,-1
3,what else did bernie sanders steal in today so...,-1
4,sorry this bernie sanders data breach has a fu...,-1
...,...,...
115,even on our worst days we are all better for t...,0
116,a rentcontrolled apartment in brooklyn is ever...,0
117,bernie sanders let me just say that i agree wi...,0
118,this is a mating call for a young bernie sanders,0


In [28]:
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),
])

In [32]:
X_train, X_test, y_train, y_test = train_test_split(sandersTrain['text'],
                                                    sandersTrain['s_label'],random_state = 0)

In [64]:
model = pipeline_sgd.fit(buttigiegTrain['text'], buttigiegTrain['s_label'])
y_predict = model.predict(buttigiegFull['text'])

In [66]:
y_predict

6161

In [63]:
model

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=1000,